In [1]:
import math

In [2]:
data = open('data/day10.txt', 'r').read()

In [5]:
def find_start_coords(grid):
    """
    Find the coordinates matching 'S' in the grid
    """
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            if grid[row][col] == 'S':
                return (row, col)

In [6]:
def find_valid_neighbors(grid, curr_coords):
    """
    Find all valid neighbor(s) (not '.') of the current coordinates
    """
    pipe_connections = {'|': {'N', 'S'}, '-': {'E', 'W'}, 'L': {'N', 'E'}, 'J': {'N', 'W'}, '7': {'S', 'W'}, 'F': {'S', 'E'}, 'S': {'N', 'E', 'S', 'W'}}
    movements = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}
    valid_neighbor_coords = []
    
    (curr_row, curr_col) = curr_coords
    curr_pipe = grid[curr_row][curr_col]
    neighbor_directions = pipe_connections[curr_pipe] 
    neighbor_movements = [movements[direction] for direction in neighbor_directions]
    
    for (row_movement, col_movement) in neighbor_movements:
        (neighbor_col, neighbor_row) = neighbor_coords = (curr_row + row_movement, curr_col + col_movement)
        if grid[neighbor_col][neighbor_row] != '.':
            valid_neighbor_coords.append(neighbor_coords)
    
    return valid_neighbor_coords

In [7]:
def find_start_neighbors(grid, start_coords):
    """
    Find the actual valid neighbor(s) of the starting point
    """
    # Find all valid potential neighbors of the starting coordinates
    start_neighbor_coords = find_valid_neighbors(grid, start_coords)
    # Find the actual connected neighbors
    actual_neighbors = {}
    for neighbor_coords in start_neighbor_coords:
        neighbor_coords_neighbors = find_valid_neighbors(grid, neighbor_coords)
        # Store the non-starting neighbors of actual connected neighbors
        if start_coords in neighbor_coords_neighbors:
            neighbor_coords_neighbors = list(set(neighbor_coords_neighbors) - {start_coords})
            actual_neighbors[neighbor_coords] = neighbor_coords_neighbors
    
    return actual_neighbors

In [8]:
def identify_start_pipe(grid, start_coords, start_neighbors):
    """
    Identify what the start pipe is and replace 'S' with it in the grid
    """
    pipe_connections = {'|': {'N', 'S'}, '-': {'E', 'W'}, 'L': {'N', 'E'}, 'J': {'N', 'W'}, '7': {'S', 'W'}, 'F': {'S', 'E'}, 'S': {'N', 'E', 'S', 'W'}}
    directions = {(-1, 0): 'N', (1, 0): 'S', (0, 1): 'E', (0, -1): 'W'}
    
    # Find the direction to which we travel to arrive at the neighbors
    (start_row, start_col) = start_coords
    movements_to_neighbor = [(neighbor_row - start_row, neighbor_col - start_col) for (neighbor_row, neighbor_col) in start_neighbors]
    neighbor_directions = [directions[movement] for movement in movements_to_neighbor]
    
    # Look up the starting pipe
    for pipe in pipe_connections:
        if pipe_connections[pipe] == set(neighbor_directions):
            break
    
    # Replace 'S' with the starting pipe
    grid[start_row][start_col] = pipe
    
    return grid

In [9]:
def find_loop(grid):
    """
    Find the loop
    """
    loop = []
    # Get the starting coordinates
    start_coords = find_start_coords(grid)
    loop.append(start_coords)
    # Find the neighbors connected to the starting coordinates
    actual_neighbors = find_start_neighbors(grid, start_coords)
    start_neighbors = list(find_start_neighbors(grid, start_coords).keys())
    loop += start_neighbors
    # Identify the starting pipe
    grid = identify_start_pipe(grid, start_coords, start_neighbors)
    # Find the rest of the loop
    coords_to_check = [coord for neighbor in list(actual_neighbors.values()) for coord in neighbor]
    while len(coords_to_check) != 0:
        curr_coords = coords_to_check.pop()
        loop.append(curr_coords)
        curr_neighbor_coords = find_valid_neighbors(grid, curr_coords)
        # If both of the coordinates are in the loop already, we've found the end of the loop
        if all([curr_neighbor_coord in loop for curr_neighbor_coord in curr_neighbor_coords]):
            break
        # Otherwise, continue with searching for the rest of the loop
        else:
            unchecked_neighbors = list(set(curr_neighbor_coords) - set(loop))
            coords_to_check += unchecked_neighbors
    
    return loop

In [10]:
def part1(data):
    grid = [list(line) for line in data.split('\n')]
    loop = find_loop(grid)
    steps = math.ceil((len(loop) - 1) / 2)
    
    return steps
part1(data)

7066

In [11]:
def print_grid(grid):
    """
    Print the grid
    """
    for row in range(len(grid)):
        print(''.join(grid[row]))

In [111]:
def update_non_loop_tiles(grid, loop):
    """
    Update non-loop tiles to '.'
    """
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            if (row, col) not in loop:
                grid[row][col] = '.'
    
    return grid

In [114]:
def categorize_non_loop_tiles(grid):
    """
    Categorizes non-loop tiles as 'I' (inside) or 'O' (outside) and counts enclosed tiles
    
    Used a hint from Reddit for the boundary line count method
    """
    num_enclosed = 0
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            if grid[row][col] == '.':
                lines_crossed = 0
                curr_col = col
                # Count the # of times the point crosses over a vertical line going to the left boundary
                while curr_col >= 0:
                    if grid[row][curr_col] in ['J', '|', 'L']:
                        lines_crossed += 1
                    curr_col -= 1
                # If an even # of lines are crossed, it's outside
                if lines_crossed % 2 == 0:
                    grid[row][col] = 'O'
                # If an odd # of lines are crossed, it's inside
                else:
                    num_enclosed += 1
                    grid[row][col] = 'I'
    
    return grid, num_enclosed

In [113]:
def part2(data):
    grid = [list(line) for line in data.split('\n')]
    loop = find_loop(grid)
    updated_grid = update_non_loop_tiles(grid, loop)
    final_grid, num_enclosed = categorize_non_loop_tiles(updated_grid)
    
    return final_grid, num_enclosed

final_grid, num_enclosed = part2(data)
num_enclosed

401